In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

import libStep1 as s1
import libStep2 as s2
import libFileio as f
import importlib


from tqdm import tqdm

In [5]:
importlib.reload(f)
importlib.reload(s1)
importlib.reload(s2)

srcDataDir = 'data/v7/'
vNum = '7'

# Load Data

In [8]:
# load all cam data
data_name = 'cam1234_lane3_combined_v' + vNum
data = f.load_data(srcDataDir +  data_name + '.mat', scalePos=False)

# get vehicle ID list
vidList = np.unique(data[:,0])
print('There are ' + str(len(vidList)) + ' vehicles in this dataset.')



Data position are ONLY converted to meter
There are 443 vehicles in this dataset.


# Trajectory Fusing and Motion Estimation

In [4]:
# combine cam and motion est
startTime = time.time() # time run time
dataX = np.zeros((1,18))
for vid in tqdm(vidList):
    # get veh
    dataVeh = f.get_veh(data, vid)
    # est motion
    dataTemp = s1.combine_cam_motion_est_ud(dataVeh)

    dataX = np.concatenate((dataX, dataTemp), axis=0)

    dataX = dataX[1:-1,:]

print('\n\nruntime = ' + str(time.time() - startTime))

f.save_data_step1(dataX, 'data/step1/', data_name + '.mat')
   

100%|██████████| 470/470 [00:21<00:00, 21.71it/s]
runtime = 21.65142059326172
Saved to data/step1/lcv_cam1234_4567_v7.mat


# Stop Handling

In [8]:
##########################################################
## Start step 2
data = dataX.copy()

#### 
# get vehicle ID list
vidList = np.unique(data[:,0])
# print('There are ' + str(len(vidList)) + ' vehicles in this dataset.')

startTime = time.time() # time run time

#################################
##### step 2
dataS = data.copy()
stopInfoAll = np.zeros((1,5))
for vid in tqdm(vidList):
    ind = dataS[:,0]==vid
    dataVehS, isStopped, stopInfo = s2.ns_and_s_handle(dataS[ind, :])
    if isStopped:
        dataS[ind, :] = dataVehS
        stopInfoAll = np.vstack((stopInfoAll, stopInfo))
        
stopInfoAll = stopInfoAll[1:,:]

print('\n\nruntime = ' + str(time.time() - startTime))

f.save_data_step2(dataS, 'data/step2/', data_name + '_op.mat')
f.save_data(stopInfoAll, 'data/step2/', data_name + '_op_stop_info.mat', "stopInfo")

100%|██████████| 470/470 [00:00<00:00, 926.81it/s]

runtime = 0.51511549949646
Saved to data/step2/lcv_cam1234_4567_v7_op.mat
Saved to data/step2/lcv_cam1234_4567_v7_op_stop_info.mat

